In [1]:
#!/usr/bin/env python
import pandas as pd
import scipy.io.wavfile as wf
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
#import scipy.io.m4afile as m4a
import glob
import scipy.io
import librosa
import numpy as np

In [2]:
class VoiceActivityDetector():
    """ Use signal energy to detect voice activity in wav file """
    
    def __init__(self, wave_input_filename):
        self._read_wav(wave_input_filename)._convert_to_mono()
        self.sample_window = 0.02 #20 ms
        self.sample_overlap = 0.01 #10ms
        self.speech_window = 0.5 #half a second
        self.speech_energy_threshold = 0.6 #60% of energy in voice band
        self.speech_start_band = 300
        self.speech_end_band = 3000
           
    def _read_wav(self, wave_file):
        self.rate, self.data = wf.read(wave_file)
        self.channels = len(self.data.shape)
        self.filename = wave_file
        return self
    
    def _convert_to_mono(self):
        if self.channels == 2 :
            self.data = np.mean(self.data, axis=1, dtype=self.data.dtype)
            self.channels = 1
        return self
    
    def _calculate_frequencies(self, audio_data):
        data_freq = np.fft.fftfreq(len(audio_data),1.0/self.rate)
        data_freq = data_freq[1:]
        return data_freq    
    
    def _calculate_amplitude(self, audio_data):
        data_ampl = np.abs(np.fft.fft(audio_data))
        data_ampl = data_ampl[1:]
        return data_ampl
        
    def _calculate_energy(self, data):
        data_amplitude = self._calculate_amplitude(data)
        data_energy = data_amplitude ** 2
        return data_energy
        
    def _znormalize_energy(self, data_energy):
        energy_mean = np.mean(data_energy)
        energy_std = np.std(data_energy)
        energy_znorm = (data_energy - energy_mean) / energy_std
        return energy_znorm
    
    def _connect_energy_with_frequencies(self, data_freq, data_energy):
        energy_freq = {}
        for (i, freq) in enumerate(data_freq):
            if abs(freq) not in energy_freq:
                energy_freq[abs(freq)] = data_energy[i] * 2
        return energy_freq
    
    def _calculate_normalized_energy(self, data):
        data_freq = self._calculate_frequencies(data)
        data_energy = self._calculate_energy(data)
        #data_energy = self._znormalize_energy(data_energy) #znorm brings worse results
        energy_freq = self._connect_energy_with_frequencies(data_freq, data_energy)
        return energy_freq
    
    def _sum_energy_in_band(self,energy_frequencies, start_band, end_band):
        sum_energy = 0
        for f in energy_frequencies.keys():
            if start_band<f<end_band:
                sum_energy += energy_frequencies[f]
        return sum_energy
    
    def _median_filter (self, x, k):
        assert k % 2 == 1, "Median filter length must be odd."
        assert x.ndim == 1, "Input must be one-dimensional."
        k2 = (k - 1) // 2
        y = np.zeros ((len (x), k), dtype=x.dtype)
        y[:,k2] = x
        for i in range (k2):
            j = k2 - i
            y[j:,i] = x[:-j]
            y[:j,i] = x[0]
            y[:-j,-(i+1)] = x[j:]
            y[-j:,-(i+1)] = x[-1]
        return np.median (y, axis=1)
        
    def _smooth_speech_detection(self, detected_windows):
        median_window=int(self.speech_window/self.sample_window)
        if median_window%2==0: median_window=median_window-1
        median_energy = self._median_filter(detected_windows[:,1], median_window)
        return median_energy
        
    def convert_windows_to_readible_labels(self, detected_windows):
        """ Takes as input array of window numbers and speech flags from speech
        detection and convert speech flags to time intervals of speech.
        Output is array of dictionaries with speech intervals.
        """
        speech_time = []
        is_speech = 0
        for window in detected_windows:
            if (window[1]==1.0 and is_speech==0): 
                is_speech = 1
                speech_label = {}
                speech_time_start = window[0] / self.rate
                speech_label['speech_begin'] = speech_time_start
                print(window[0], speech_time_start)
                #speech_time.append(speech_label)
            if (window[1]==0.0 and is_speech==1):
                is_speech = 0
                speech_time_end = window[0] / self.rate
                speech_label['speech_end'] = speech_time_end
                speech_time.append(speech_label)
                print(window[0], speech_time_end)
        return speech_time
      
    def plot_detected_speech_regions(self):
        """ Performs speech detection and plot original signal and speech regions.
        """
        data = self.data
        detected_windows = self.detect_speech()
        data_speech = np.zeros(len(data))
        it = np.nditer(detected_windows[:,0], flags=['f_index'])
        while not it.finished:
            data_speech[int(it[0])] = data[int(it[0])] * detected_windows[it.index,1]
            it.iternext()
        plt.figure()
        plt.plot(data_speech)
        plt.plot(data)
        plt.show()
        return self
       
    def detect_speech(self):
        """ Detects speech regions based on ratio between speech band energy
        and total energy.
        Output is array of window numbers and speech flags (1 - speech, 0 - nonspeech).
        """
        detected_windows = np.array([])
        sample_window = int(self.rate * self.sample_window)
        sample_overlap = int(self.rate * self.sample_overlap)
        data = self.data
        sample_start = 0
        start_band = self.speech_start_band
        end_band = self.speech_end_band
        while (sample_start < (len(data) - sample_window)):
            sample_end = sample_start + sample_window
            if sample_end>=len(data): sample_end = len(data)-1
            data_window = data[sample_start:sample_end]
            energy_freq = self._calculate_normalized_energy(data_window)
            sum_voice_energy = self._sum_energy_in_band(energy_freq, start_band, end_band)
            sum_full_energy = sum(energy_freq.values())
            speech_ratio = sum_voice_energy/sum_full_energy
            # Hipothesis is that when there is a speech sequence we have ratio of energies more than Threshold
            speech_ratio = speech_ratio>self.speech_energy_threshold
            detected_windows = np.append(detected_windows,[sample_start, speech_ratio])
            sample_start += sample_overlap
        len_detected_windows =  len(detected_windows)//2
        print(len_detected_windows)
        detected_windows = detected_windows.reshape(len_detected_windows,2)
        detected_windows[:,1] = self._smooth_speech_detection(detected_windows)
        return detected_windows

In [3]:
def entropy_SVD(data,window_size):
    df = []
    for i in range(int(len(data)/window_size)):
        complexity = nk.complexity(data[i*window_size:(i+1)*window_size])
        df.append(complexity['Entropy_SVD'])
    df = pd.DataFrame(df)
    return df

In [14]:
def spectral_entropy(data,window_size):
    df = []
    for i in range(int(len(data)/window_size)):
        complexity = nk.complexity(data[i*window_size:(i+1)*window_size],spectral=True,shannon=False,sampen=False,multiscale=False,svd = False,correlation=False,
                          higushi = False,petrosian=False,fisher = False,hurst = False,dfa= False,lyap_r=False,lyap_e =False)
        df.append(complexity['Entropy_Spectral'])
    df = pd.DataFrame(df)
    return df

In [5]:
def kurtosis(data,window_size):
    #data = pd.DataFrame(data)
    df=[]
    for i in range(int(len(data)/window_size)):
        df.append(data[i*window_size:(i+1)*window_size].kurtosis())
    df = pd.DataFrame(df)
    return df

In [6]:
def average(data,window_size):
    df = pd.DataFrame()
    for i in range(int(len(data)/window_size)):
        dt = np.array(data[i*window_size:(i+1)*window_size])
        window = np.ones((window_size,))/float(window_size)
        dt = pd.DataFrame(np.convolve(dt, window,mode='valid'))
        df = pd.concat([df,dt])
    return df

In [7]:
def root_mean_square(data,window_size):
    df = pd.DataFrame()
    for i in range(int(len(data)/window_size)):
        dt = np.power(data[i*window_size:(i+1)*window_size],2)
        window = np.ones((window_size,))/float(window_size)
        dt = pd.DataFrame(np.sqrt(np.convolve(dt, window,mode='valid')))
        df = pd.concat([df,dt])
    return df

In [ ]:
name_file = 'EEG .mat files/u_Filters.mat' #put the file .mat name
audio_file = 'Audio wav files/mason-1-u.wav'
path = '/home/guatam-admin/Vijay_BCI/Data-Mason-1/' #put the directory
(rate,sig) = wav.read(path+audio_file)
mfcc_feat = mfcc(sig,rate,winstep=0.01)
mf = pd.DataFrame(mfcc_feat)
v = VoiceActivityDetector(path+audio_file)
label = pd.DataFrame(v.detect_speech())
label.columns = ['time','label']
if len(mf)!=len(label):
    fill_na = pd.DataFrame({'time':[label['time'][len(label)-1]+120.0],
                       'label':[label['label'][len(label)-1]]})
    label = label.append(fill_na).reset_index(drop=True)
mat = scipy.io.loadmat(path+name_file)
mat = {k:v for k, v in mat.items() if k[0]!='_'}
names = list(mat.keys())
df = pd.DataFrame()
for key in names[:-8]:
    value = mat[key]
    value = pd.DataFrame(value)
    df = pd.merge(df,value,how='right',right_index=True,left_index=True)
df.columns = names[:-8]
df_zero = pd.DataFrame()
for name in df.columns:
    zero = librosa.feature.zero_crossing_rate(df[name].values,frame_length=100,hop_length=10)
    zero = pd.DataFrame(np.transpose(zero))
    df_zero = pd.merge(df_zero,zero,how='right',right_index=True,left_index=True)
df_zero.columns = [column+'_zero' for column in df.columns]
df_zero = df_zero*int(100)
df_avg = pd.DataFrame()
for name in df.columns:
    avg = average(df[name].values,window_size=10)
    df_avg = pd.concat([df_avg,avg],axis=1)
df_avg.columns = [column+'_avg' for column in df.columns]
df_rms = pd.DataFrame()
for name in df.columns:
    rms = root_mean_square(df[name].values,window_size=10)
    df_rms = pd.concat([df_rms,rms],axis=1)
df_rms.columns = [column+'_rms' for column in df.columns]
df_spectral = pd.DataFrame()
for name in df.columns:
    spectral = spectral_entropy(df[name].values,window_size=10)
    #rms = pd.DataFrame(np.transpose(zero))
    df_spectral = pd.concat([df_spectral,spectral],axis=1)
df_spectral.columns = [column+'_spectral' for column in df.columns]
df_kurt = pd.DataFrame()
for name in df.columns:
    kurt = kurtosis(df[name],window_size=10)
    #rms = pd.DataFrame(np.transpose(zero))
    df_kurt = pd.concat([df_kurt,kurt],axis=1)
df_kurt.columns = [column+'_kurt' for column in df.columns]
df_rms = df_rms.reset_index(drop=True)
df_zero = df_zero.reset_index(drop=True)
df_avg = df_avg.reset_index(drop=True)
df_kurt = df_kurt.reset_index(drop=True)
df_spectral = df_spectral.reset_index(drop=True)
df_kurt = df_kurt[(len(df_kurt)-len(mf)):len(df_kurt)].reset_index(drop=True)
df_spectral = df_spectral[(len(df_spectral)-len(mf)):len(df_spectral)].reset_index(drop=True)
df_rms = df_rms[(len(df_rms)-len(mf)):len(df_rms)].reset_index(drop=True)
df_zero = df_zero[(len(df_zero)-len(mf)):len(df_zero)].reset_index(drop=True)
df_avg = df_avg[(len(df_avg)-len(mf)):len(df_avg)].reset_index(drop=True)
final_data = pd.DataFrame()
final_data = pd.concat([df_rms,df_zero,df_avg,df_spectral,df_kurt,label],axis=1)
final_data = final_data.drop(['time'],axis=1)
final_data.to_csv('/home/guatam-admin/Vijay_BCI/Vowel_data/Mason/final_u.csv',index=False)

30119


In [103]:
final_data.to_csv('/home/guatam-admin/Vijay_BCI/Vowel_data/Mason/final_i.csv',index=False)

In [90]:
fill_na = pd.DataFrame({'time':[label['time'][len(label)-1]+120.0],
                       'label':[label['label'][len(label)-1]]})

In [97]:
final_data = final_data.fillna(0)

In [106]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30053 entries, 0 to 30052
Columns: 156 entries, Fp1_rms to label
dtypes: float64(156)
memory usage: 35.8 MB
